In [1]:
import json, os
import numpy as np
import random, math
import copy

def read_json(fname, key_int=False):
    with open(fname, 'r') as file:
        data = file.read()
        json_data = json.loads(data)
        
        if not key_int:
            return json_data
        
        json_data = {int(key): value for key, value in json_data.items()}
        return json_data
    
CHAR_INDICES = read_json('../models/CHAR_INDICES.json', key_int=False)

In [2]:
import glob
import random

file_list = glob.glob('../sastra/*.txt', recursive=True)
print('Total files:', len(file_list))

# get only 20 files
file_list = file_list[:13]

#train val test size
train_size = 0.7
val_size = 0.15
test_size = 0.15

#splitting the data
# random.shuffle(file_list)
# train_list = file_list[:int(len(file_list)*train_size)]
# val_list = file_list[int(len(file_list)*train_size):int(len(file_list)*(train_size+val_size))]
# test_list = file_list[int(len(file_list)*(train_size+val_size)):]

# train_list = ['../sastra/2682.txt', '../sastra/1704.txt', '../sastra/2565.txt', '../sastra/1537.txt', '../sastra/1903.txt', '../sastra/1260.txt', '../sastra/2732.txt', '../sastra/1910.txt', '../sastra/220.txt', '../sastra/2224.txt', '../sastra/2566.txt', '../sastra/1403.txt', '../sastra/1906.txt', '../sastra/1447.txt', '../sastra/2810.txt', '../sastra/224.txt', '../sastra/3030.txt', '../sastra/2225.txt', '../sastra/1909.txt', '../sastra/2571.txt', '../sastra/2685.txt', '../sastra/2568.txt', '../sastra/2812.txt', '../sastra/1900.txt', '../sastra/1908.txt', '../sastra/221.txt', '../sastra/1538.txt', '../sastra/2770.txt', '../sastra/2570.txt', '../sastra/2567.txt', '../sastra/1534.txt', '../sastra/2813.txt', '../sastra/1462.txt', '../sastra/2811.txt', '../sastra/1902.txt', '../sastra/2684.txt', '../sastra/1463.txt', '../sastra/2681.txt', '../sastra/2733.txt', '../sastra/3032.txt']
# val_list = ['../sastra/2687.txt', '../sastra/2564.txt', '../sastra/3031.txt', '../sastra/1901.txt', '../sastra/2683.txt', '../sastra/1904.txt', '../sastra/1907.txt', '../sastra/219.txt', '../sastra/1657.txt']
# test_list = ['../sastra/2686.txt', '../sastra/222.txt', '../sastra/1535.txt', '../sastra/2569.txt', '../sastra/1539.txt', '../sastra/1536.txt', '../sastra/1437.txt', '../sastra/1533.txt', '../sastra/1905.txt']

# train_list = ['../sastra/1463.txt', '../sastra/1538.txt', '../sastra/1534.txt', '../sastra/1901.txt', '../sastra/1437.txt', '../sastra/1539.txt', '../sastra/1537.txt', '../sastra/1903.txt', '../sastra/1403.txt', '../sastra/1260.txt', '../sastra/1536.txt', '../sastra/1657.txt', '../sastra/1447.txt', '../sastra/1900.txt']
# val_list = ['../sastra/1533.txt', '../sastra/1462.txt', '../sastra/1535.txt']
# test_list = ['../sastra/1902.txt', '../sastra/1904.txt', '../sastra/1704.txt']

train_list = ['../sastra/1437.txt', '../sastra/1537.txt', '../sastra/1533.txt', '../sastra/1447.txt', '../sastra/1539.txt', '../sastra/1403.txt', '../sastra/1535.txt', '../sastra/1463.txt', '../sastra/1538.txt']
val_list =  ['../sastra/1260.txt', '../sastra/1536.txt']
test_list =  ['../sastra/1534.txt', '../sastra/1462.txt']

print('Train size:', len(train_list))
print('Val size:', len(val_list))
print('Test size:', len(test_list))

print("TRAIN FILES:")
print(train_list)

print("VAL FILES:")
print(val_list)

print("TEST FILES:")
print(test_list)

# compute total char count in each file
def get_total_char_count(file_list):
    total_char_count = 0
    for file in file_list:
        with open(file, 'r') as f:
            total_char_count += len(f.read())
    return total_char_count

train_total_char_count = get_total_char_count(train_list)
val_total_char_count = get_total_char_count(val_list)
test_total_char_count = get_total_char_count(test_list)

print('Train total char count:', train_total_char_count)
print('Val total char count:', val_total_char_count)
print('Test total char count:', test_total_char_count)

# train %
train_char_percentage = train_total_char_count / (train_total_char_count + val_total_char_count + test_total_char_count)
val_char_percentage = val_total_char_count / (train_total_char_count + val_total_char_count + test_total_char_count)
test_char_percentage = test_total_char_count / (train_total_char_count + val_total_char_count + test_total_char_count)

print('Train char percentage:', train_char_percentage)
print('Val char percentage:', val_char_percentage)
print('Test char percentage:', test_char_percentage)


Total files: 13
Train size: 9
Val size: 2
Test size: 2
TRAIN FILES:
['../sastra/1437.txt', '../sastra/1537.txt', '../sastra/1533.txt', '../sastra/1447.txt', '../sastra/1539.txt', '../sastra/1403.txt', '../sastra/1535.txt', '../sastra/1463.txt', '../sastra/1538.txt']
VAL FILES:
['../sastra/1260.txt', '../sastra/1536.txt']
TEST FILES:
['../sastra/1534.txt', '../sastra/1462.txt']
Train total char count: 612172
Val total char count: 149084
Test total char count: 161806
Train char percentage: 0.6631970550190561
Val char percentage: 0.16151027774949028
Test char percentage: 0.17529266723145356


In [5]:
# CREATE DICTIONARY

dict_path = "../models/dictionary.txt"
def create_dict(file_list):
    words = []
    for file in file_list:
        with open(file, 'r') as f:
            delimeter = "|"
            for line in f:
                line = line.strip()
                line = line.split(delimeter)
                for word in line:
                    if word not in words:
                        words.append(word)


    with open(dict_path, 'w') as f:
        for word in words:
            f.write(word + '\n')

    return words

def read_dict():
    with open(dict_path, 'r') as f:
        words = f.read().splitlines()
    return words

# create dictionary
file_list = train_list + val_list
create_dict(file_list)

DICT = read_dict()

In [6]:
from tqdm import tqdm

def replace(text, wordlist):
    for new_word, list_ in wordlist.items():
        for old_word in list_:
            text = text.replace(old_word.lower(), new_word)
    
    return text

def preprocessing(text, wordlist):
    text = text.lower()  # to lower-case
    text = replace(text, wordlist)

    # check if char is in CHAR_INDICES
    for char in text:
        if char not in CHAR_INDICES:
            text = text.replace(char, '')
            
    return text

def generate_dict_vector(text):
    vect = [0.0] * len(text)
    for i in range(len(text)):
        current_text = text[:i+1]
        if current_text in DICT:
            vect[i] = 1.0
        else:
            vect[i] = 0.0

    return vect

def create_dataset(text, look_back, look_front, file_name, type_):
    X, y = [], []
    X_ngram = []
    text = '|' + text
    data = ['?'] * look_back

    directory = f"../generated/{type_}"
    if not os.path.exists(directory):
        os.makedirs(directory)

    dataset_path = f"../generated/{type_}/{file_name}.txt"
    progress_bar = tqdm(total=len(text), desc="Processing")

    for i in range(1, len(text)):
        current_char = text[i]
        before_char = text[i-1]

        if current_char == '|':
            continue

        data = data[1:look_back] + [current_char]  # X data
        target = 1 if before_char == '|' else 0  # y data

        # look front
        front_text = current_char
        j = 0
        while len(data) < look_back + look_front:
            if i+j+1 < len(text):
                if text[i+j+1] == '|':
                    j += 1
                    continue
                data.append(text[i+j+1])
                front_text += text[i+j+1]
            else:
                data.append('?')
                front_text += '?'

            j += 1
        
        ngram_vector = generate_dict_vector(front_text)
        
        str_data = '|'.join(data)
        str_ngram_vector = '|'.join([str(x) for x in ngram_vector])
        str_target = str(target)
        # print(data, ngram_vector, target) 

        # print(str_data + '\t' + str_ngram_vector + '\t' + str_target)
        with open(dataset_path, 'a') as f:
            f.write(str_data + '\t' + str_ngram_vector + '\t' + str_target + '\n')

        progress_bar.update(1)
        
    # y[-1] = 1  # end text is '|'
    # return X, X_ngram, y
    return None, None, None


def encode_data(X, y):
    sequence_len = len(X[0])
    encode_X = np.zeros((len(X), sequence_len, len(CHAR_INDICES)), dtype=np.float32)
    encode_y = np.zeros((len(y), 2), dtype=np.float32)
    for i, (data, target) in enumerate(zip(X, y)):
        for t, char in enumerate(data):
            encode_X[i, t, char] = 1
        encode_y[i, target] = 1
        
    return encode_X, encode_y


def read_text(file):
    return open(file, encoding='utf-8', errors="ignore").read()

In [7]:
def replace(text, wordlist):
    for new_word, list_ in wordlist.items():
        for old_word in list_:
            text = text.replace(old_word.lower(), new_word)
    
    return text

def preprocessing(text, wordlist):
    text = text.lower()  # to lower-case
    text = replace(text, wordlist)

    # check if char is in CHAR_INDICES
    for char in text:
        if char not in CHAR_INDICES:
            text = text.replace(char, '')
            
    return text

wordlist = {
    '': ['\n'],
}

wordlist2 = copy.deepcopy(wordlist)
wordlist2[''].append('|')


In [10]:
# TRAIN DATASET
files = train_list
look_back = 10
look_front = 5
copy_files = files.copy()
random.shuffle(copy_files)

for file in copy_files:
    print(f'File: {file}')
    file_name = file.split('/')[-1].split('.')[0]

    text = read_text(file)
    text = preprocessing(text, wordlist)
    create_dataset(text, look_back, look_front, file_name, 'train')

File: ../sastra/1463.txt


Processing:  86%|████████▌ | 870/1011 [00:03<00:00, 240.10it/s]


File: ../sastra/1403.txt


Processing:  83%|████████▎ | 17138/20557 [02:19<00:27, 123.17it/s]


File: ../sastra/1437.txt


Processing:  85%|████████▍ | 5884/6955 [01:11<00:12, 82.51it/s] 


File: ../sastra/1538.txt


Processing:  84%|████████▎ | 70651/84463 [08:16<01:37, 142.27it/s]


File: ../sastra/1537.txt


Processing:  57%|█████▋    | 42242/74050 [04:55<03:52, 137.05it/s]

In [9]:
# VALIDATION DATASET
files = val_list
look_back = 10
look_front = 5
copy_files = files.copy()
random.shuffle(copy_files)

for file in copy_files:
    print(f'File: {file}')
    file_name = file.split('/')[-1].split('.')[0]

    text = read_text(file)
    text = preprocessing(text, wordlist)
    create_dataset(text, look_back, look_front, file_name, 'val')

File: ../sastra/1260.txt
File: ../sastra/1536.txt


In [ ]:
# TEST DATASET
files = test_list
look_back = 10
look_front = 5
copy_files = files.copy()
random.shuffle(copy_files)

for file in copy_files:
    print(f'File: {file}')
    file_name = file.split('/')[-1].split('.')[0]

    text = read_text(file)
    text = preprocessing(text, wordlist)
    create_dataset(text, look_back, look_front, file_name, 'test')